In [ ]:
from itertools import product

import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from matplotlib.patches import Rectangle
from uncertainties import ufloat
from uncertainties import unumpy as unp

import dlqmc.mplext
from deepqmc.ewm import EWMAverage
from deepqmc.wf.paulinet import DistanceBasis

In [ ]:
# needs to be in a separate cell, see https://github.com/ipython/ipython/issues/11098
mpl.rcParams['figure.dpi'] = 150
mpl.rc('font', family='serif', serif='STIXGeneral', size=9)
mpl.rc('mathtext', fontset='stix')
mpl.rc('axes', titlesize=9)
COLORS = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
def savefig(fig, name, ext='pdf', **kwargs):
    fig.savefig(
        f'../pub/figs/{name}.{ext}',
        transparent=True,
        bbox_inches='tight',
        pad_inches=0,
        **kwargs,
    )


def to_corr(x, ref):
    return (ref[0] - x) / (ref[0] - ref[1])


def to_corr_error(x, ref):
    return x / (ref[0] - ref[1])

## Learning curves

In [ ]:
systems = ['H2', 'Be', 'B', 'C', 'LiH', 'Li2']
ansatzes = ['SD-SJ', 'SD-SJBF', 'MD-SJ', 'MD-SJBF']
results = {}
with h5py.File('../data/raw/data_pub_small_systems.h5', 'r') as f:
    for system, ansatz in product(systems, ansatzes):
        E_mean = f[system][ansatz]['train'][...].mean(axis=1)
        ewm = EWMAverage(outlier_maxlen=3, outlier=3, decay_alpha=10)
        E_ewm = []
        for e in E_mean:
            ewm.update(e)
            E_ewm.append((ewm.mean.item().n, ewm.mean.item().s))
        E_ewm = unp.uarray(*zip(*E_ewm))
        results[system, ansatz] = (E_ewm, f[system].attrs['ref_energy'])

In [ ]:
def plot_mean_err(ax, data, ref_enes, bs, decay):
    step, E_ewm = data
    inds = np.geomspace(1, 9_999, 200).astype(int)
    energy = to_corr(unp.nominal_values(E_ewm), ref_enes)
    err = to_corr_error(unp.std_devs(E_ewm), ref_enes)
    ax.plot(step[inds], energy[inds])
    ax.fill_between(
        step[inds], (energy + err)[inds], (energy - err)[inds], color='grey', alpha=0.5
    )


fig, axes = plt.subplots(
    len(systems),
    len(ansatzes),
    figsize=(3.6, 5.5),
    gridspec_kw=dict(hspace=0.08, wspace=0.09),
    sharex=True,
    sharey=True,
)
for (i, system), (j, ansatz) in product(enumerate(systems), enumerate(ansatzes),):
    ax = axes[i, j]
    plot_mean_err(
        ax,
        (np.arange(0, 10_000), results[system, ansatz][0]),
        results[system, ansatz][1],
        10_000,
        3,
    )
    ax.set_xscale('log')
    ax.set_yscale('corr_energy')
    ax.set_xticks([1, 10, 100, 1000])
    ax.set_xticklabels([1, None, None, 1000])
    ax.xaxis.set_minor_locator(mpl.ticker.LogLocator(subs=(4, 7), numticks=8))
    ax.set_xticklabels([], minor=True)
    ax.set_xlim(1, 1e4)
    ax.set_ylim(-0.3, 0.999)
    ax.grid(axis='y', which='major', ls='dotted')
    if i == 0:
        ax.set_title(ansatz)
    if j == 0:
        ax.set_ylabel(fr'$\mathrm{{{system.replace("2", "_2")}}}$', labelpad=16)
fig.text(0.5, 0.06, 'iterations', ha='center')
fig.text(-0.015, 0.5, 'correlation energy', rotation='vertical', va='center')
savefig(fig, 'learning-curves')

## H10

In [ ]:
distances = np.array([1.2, 1.4, 1.6, 1.8, 2.0, 2.4, 2.8, 3.2, 3.6])
systems = [f'H10_d{di}' for di in distances]
ansatzes = ['SD-SJ', 'SD-SJBF', 'MD-SJBF']
data_corr = {}
with h5py.File('../data/raw/data_pub_h10.h5', 'r') as f:
    ref_energies = np.array([f[system].attrs['ref_energy'] for system in systems])
    for ansatz in ansatzes:
        data = np.array([f[system][ansatz].attrs['energy'] for system in systems])
        data_corr[ansatz] = np.array(
            [
                [to_corr(ei[0], ei_ref), to_corr_error(ei[1], ei_ref)]
                for ei, ei_ref in zip(data, ref_energies)
            ]
        )

In [ ]:
fig, ax = plt.subplots(figsize=(3.63, 2.7))
for i, ansatz in enumerate(ansatzes):
    ax.errorbar(
        distances,
        data_corr[ansatz][:, 0],
        data_corr[ansatz][:, 1],
        label=ansatz,
        ls=[':', '-.', '-'][i],
        fillstyle=['none', 'full', 'full'][i],
        marker='o',
        ms='4',
        color='C0',
    )

ax.legend(loc='lower center', bbox_to_anchor=(0.43, 1), ncol=3)
ax.set_ylabel('correlation energy', labelpad=7)
ax.set_xlabel('H–H distance [a.u.]')
ax.set_yscale('corr_energy')
ax.set_xticks(distances[::2])
ax.grid(axis='y', which='major', ls='dotted')
savefig(fig, 'h10-dis-curve')

## Distance basis

In [ ]:
fig, ax = plt.subplots(figsize=(2, 2))
x = torch.linspace(0, 12, 300)
ax.plot(x.numpy(), DistanceBasis(32, envelope='nocusp')(x).numpy())
ax.set_xlabel(r'$r/a_0$')
ax.set_ylabel(r'$\mathbf{e}(r)$')
ax.set_yticks([0, 0.4])
savefig(fig, 'dist-features')

## Diatomics

In [ ]:
refs_qmc = pd.read_csv('../data/extern/diatomics-qmc.csv').set_index(['system', 'ref'])
refs_exact = pd.read_csv('../data/extern/diatomics-exact.csv').set_index(['system'])

dets = [1, 3, 10, 30, 100]
systems = ['Li2', 'Be2', 'B2', 'C2']
refs = ['Filippi', 'Toulouse', 'Morales']
with h5py.File('../data/raw/data_pub_diatomics.h5', 'r') as f:
    Es = np.array(
        [[f[system][f'{d}det'].attrs['energy'] for d in dets] for system in systems]
    )
    ref_energies = np.array([f[system].attrs['ref_energy'] for system in systems])

In [ ]:
def plot_vmc_dmc(ax, dets, e_vmc, e_dmc, e_ref, label, color, marker):
    ax.plot(
        dets,
        to_corr(e_vmc, e_ref),
        label=f'Ref. [{label}]',
        ls=' ',
        fillstyle='none',
        marker=marker,
        color=color,
        ms=5,
    )
    ax.plot(
        dets, to_corr(e_dmc, e_ref), ls=' ', marker='x', fillstyle='none', color=color,
    )
    ax.plot(
        [dets, dets],
        [to_corr(e_vmc, e_ref), to_corr(e_dmc, e_ref)],
        color=color,
        ls=':',
    )


fig, axes = plt.subplots(
    2,
    2,
    sharex=True,
    sharey=True,
    figsize=(3.5, 2.6),
    gridspec_kw=dict(hspace=0.08, wspace=0.06),
)
for i, ax, s, ek in zip(range(4), axes.flat, systems, Es):
    ax.errorbar(
        dets,
        to_corr(ek[:, 0], ref_energies[i]),
        to_corr_error(ek[:, 1], ref_energies[i]),
        ms=5,
        marker='o',
        ls='',
        color='C0',
        linewidth=2,
        label='PauliNet',
    )
    ax.plot(
        dets,
        to_corr(ek[:, 0], ref_energies[i]),
        ls=':',
        color='grey',
        linewidth=2,
        zorder=0,
    )
    for ref, color, marker in zip(refs, 'gry', 'o^^'):
        ref_j = refs_qmc.loc(0)[systems[i], ref]
        plot_vmc_dmc(
            ax,
            ref_j['ndet'],
            ref_j['e_vmc'],
            ref_j['e_dmc'],
            refs_exact.loc()[s],
            'abc'[refs.index(ref)],
            color,
            marker,
        )
    ax.set_xscale('log')
    ax.set_xlim(0.5, 5_000)
    ax.set_xticks([1, 10, 100, 1000])
    ax.set_xticklabels([1, 10, 100, 1000])
    ax.xaxis.set_minor_locator(mpl.ticker.LogLocator(subs=(4, 7), numticks=8))
    ax.set_xticklabels([], minor=True)
    ax.set_yscale('corr_energy')
    ax.set_ylim(0.7, 0.9992)
    ax.grid(axis='y', which='major', ls='dotted')
    ax.annotate(
        fr'$\mathrm{{{s.replace("2", "_2")}}}$', (0.05, 0.8), xycoords='axes fraction'
    )
    if ax is axes.flat[0]:
        ax.legend(
            loc='lower center',
            bbox_to_anchor=(0.8, 1),
            ncol=4,
            handletextpad=0.5,
            columnspacing=1,
        )
fig.text(
    0.5, -0.02, 'number of determinants/CSFs', ha='center', va='center',
)
fig.text(
    -0.02, 0.5, 'correlation energy', ha='center', va='center', rotation='vertical',
)
savefig(fig, 'diatomics')

## Determinants

- NN-QMC: 1s to 10s
- standard MD-QMC: 100s to 100,000s
- NN-CI: 100,000s
- FCI-QMC: 1,000,000s to 100,000,000s
- FCI: to 100,000,000s to 1,000,000,000s

In [ ]:
def sstep(x):
    return np.piecewise(
        x,
        [x <= 0, x < 1],
        [0, lambda x: -20 * x ** 7 + 70 * x ** 6 - 84 * x ** 5 + 35 * x ** 4, 1],
    )


def get_bar(ws, vmax=1, dens=100):
    return np.hstack(
        [
            np.linspace(0, vmax, dens * ws[0]),
            vmax * np.ones(dens * ws[1]),
            np.linspace(vmax, 0, dens * ws[2]),
        ]
    )


fig, ax = plt.subplots(figsize=(2.7, 1.6))
payload = [
    ('multideterminant\nQMC + NNs', 2, 50),
    ('multideterminant QMC', 100, 1e5),
    ('configuration\ninteraction + NNs', 1e5, 1e6),
    ('configuration interaction', 2e6, 2e9),
]
for i, (_, fro, to) in enumerate(payload):
    ax.add_patch(Rectangle((fro, i + 0.1), to - fro, 0.8, color='grey'))
ax.set_xlim(1, 1e10)
ax.set_ylim(0, 4)
ax.set_xscale('log')
ax.set_xlabel('number of determinants')
ax.axvline(1e5, color='black', ls='dashed')
ax.text(2.5, 4.3, '1st quantization', fontstyle='italic')
ax.text(2.0e5, 4.3, '2nd quantization', fontstyle='italic')
ax.set_yticks([0.5, 1.5, 2.5, 3.5])
ax.set_yticklabels([l for l, *_ in payload], ha='right')
savefig(fig, 'ndets')

## Cyclobutadiene

In [ ]:
results = pd.read_csv('../data/final/cyclobutadiene-fit.csv')

In [ ]:
fig, ax = plt.subplots(figsize=(3, 1.7))
for (batch, state), traj in results.groupby(['batch', 'state']):
    if batch != 250:
        continue
    color = {'ground': COLORS[0], 'transition': 'lightskyblue'}[state]
    if state == 'ground':
        state = 'minimum'
    ax.plot(traj['step'], traj['energy_ewm'], label=state, color=color)
ax.set_ylim(-154.65, None)
ax.set_xlim(10, None)
ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.5))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.1))
ax.grid(axis='y', which='major')
ax.grid(axis='y', which='minor', ls='dotted')
ax.set_xscale('log')
ax.set_xlabel('iterations')
ax.set_ylabel('total energy [a.u.]')
ax.legend()
savefig(fig, 'cyclobutadiene-training')

In [ ]:
results = pd.read_csv('../data/final/cyclobutadiene-sample.csv')
(
    results.groupby(['batch', 'state'])
    .apply(lambda x: ufloat(x['energy'].mean(), x['energy'].std() / np.sqrt(len(x))))
    .unstack()
    .pipe(lambda x: 632 * (x['ground'] - x['transition']))
)